<a href="https://colab.research.google.com/github/gundamp/NBA_Analytics/blob/main/NBA_data_analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##NBA Analytics
- Back-to-back games win/loss
- Scores mean and standard deviation

###Read data and apply some basic filters

In [ ]:
import numpy as np 
import pandas as pd
import re

import datetime as dt
from datetime import datetime

!pip install pyspark
import pyspark
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.sql import SparkSession

from google.colab import data_table
data_table.enable_dataframe_formatter()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path_games = "/content/drive/MyDrive/NBA/game_CC.csv"
path_teams = "/content/drive/MyDrive/NBA/teams.csv"

data_raw_games = pd.read_csv(path_games, encoding = 'ISO-8859-1')
data_raw_teams = pd.read_csv(path_teams, encoding = 'ISO-8859-1')

In [ ]:
#data_raw_games.head()
data_raw_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62361 entries, 0 to 62360
Data columns (total 54 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   season_id               62361 non-null  int64  
 1   team_id_home            62361 non-null  int64  
 2   team_abbreviation_home  62361 non-null  object 
 3   team_name_home          62361 non-null  object 
 4   game_id                 62361 non-null  int64  
 5   game_date               62361 non-null  object 
 6   matchup_home            62361 non-null  object 
 7   wl_home                 62354 non-null  object 
 8   min                     62361 non-null  int64  
 9   fgm_home                62343 non-null  float64
 10  fga_home                44499 non-null  float64
 11  fg_pct_home             44460 non-null  float64
 12  fg3m_home               46708 non-null  float64
 13  fg3a_home               41444 non-null  float64
 14  fg3_pct_home            41085 non-null

In [ ]:
data_raw_games.head()

,season_id,team_id_home,team_abbreviation_home,team_name_home,game_id,game_date,matchup_home,wl_home,min,fgm_home,...,dreb_away,reb_away,ast_away,stl_away,blk_away,tov_away,pf_away,pts_away,plus_minus_away,video_available_away
0,21946,1610610035,HUS,Toronto Huskies,24600001,1946-11-01 00:00:00,HUS vs. NYK,L,0,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68,2,0
1,21946,1610610034,BOM,St. Louis Bombers,24600003,1946-11-02 00:00:00,BOM vs. PIT,W,0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,25.0,51,-5,0
2,21946,1610610032,PRO,Providence Steamrollers,24600002,1946-11-02 00:00:00,PRO vs. BOS,W,0,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53,-6,0
3,21946,1610610025,CHS,Chicago Stags,24600004,1946-11-02 00:00:00,CHS vs. NYK,W,0,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,22.0,47,-16,0
4,21946,1610610028,DEF,Detroit Falcons,24600005,1946-11-02 00:00:00,DEF vs. WAS,L,0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50,17,0


In [ ]:
data_raw_games['GAME_DATE_ADJ'] = pd.to_datetime(data_raw_games['game_date'], format='%Y-%m-%d')
data_raw_games.head()

,season_id,team_id_home,team_abbreviation_home,team_name_home,game_id,game_date,matchup_home,wl_home,min,fgm_home,...,reb_away,ast_away,stl_away,blk_away,tov_away,pf_away,pts_away,plus_minus_away,video_available_away,GAME_DATE_ADJ
0,21946,1610610035,HUS,Toronto Huskies,24600001,1946-11-01 00:00:00,HUS vs. NYK,L,0,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,68,2,0,1946-11-01
1,21946,1610610034,BOM,St. Louis Bombers,24600003,1946-11-02 00:00:00,BOM vs. PIT,W,0,20.0,...,NaN,NaN,NaN,NaN,NaN,25.0,51,-5,0,1946-11-02
2,21946,1610610032,PRO,Providence Steamrollers,24600002,1946-11-02 00:00:00,PRO vs. BOS,W,0,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,53,-6,0,1946-11-02
3,21946,1610610025,CHS,Chicago Stags,24600004,1946-11-02 00:00:00,CHS vs. NYK,W,0,21.0,...,NaN,NaN,NaN,NaN,NaN,22.0,47,-16,0,1946-11-02
4,21946,1610610028,DEF,Detroit Falcons,24600005,1946-11-02 00:00:00,DEF vs. WAS,L,0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,50,17,0,1946-11-02


In [ ]:
data_raw_games['GAME_DATE_OFFSET'] = pd.DatetimeIndex(data_raw_games['GAME_DATE_ADJ']) + pd.DateOffset(-1)


###Split data so that each row represents just one team

In [ ]:
data_raw_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62361 entries, 0 to 62360
Data columns (total 56 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   season_id               62361 non-null  int64         
 1   team_id_home            62361 non-null  int64         
 2   team_abbreviation_home  62361 non-null  object        
 3   team_name_home          62361 non-null  object        
 4   game_id                 62361 non-null  int64         
 5   game_date               62361 non-null  object        
 6   matchup_home            62361 non-null  object        
 7   wl_home                 62354 non-null  object        
 8   min                     62361 non-null  int64         
 9   fgm_home                62343 non-null  float64       
 10  fga_home                44499 non-null  float64       
 11  fg_pct_home             44460 non-null  float64       
 12  fg3m_home               46708 non-null  float6

In [ ]:
data_games_frame_1 = data_raw_games.sort_values(['game_id'])
data_games_frame_2 = data_raw_games.sort_values(['game_id'])

data_games_frame_1['TEAM_ID'] = data_games_frame_1['team_id_home']
data_games_frame_2['TEAM_ID'] = data_games_frame_2['team_id_away']

data_games_frame_1['WIN_FLAG'] = (data_games_frame_1['wl_home'] == 'W')
data_games_frame_2['WIN_FLAG'] = (data_games_frame_1['wl_home'] != 'W')

data_games_frame_1['TEAM_NAME'] = data_games_frame_1['team_name_home']
data_games_frame_2['TEAM_NAME'] = data_games_frame_2['team_name_away']

data_games_frame_1['SCORE'] = data_games_frame_1['pts_home']
data_games_frame_2['SCORE'] = data_games_frame_2['pts_away']

data_games_frame_1['HOME_FLAG'] = 'Home'
data_games_frame_2['HOME_FLAG'] = 'Away'

data_games = pd.concat([data_games_frame_1, data_games_frame_2], axis = 0).drop(['team_id_home', 'team_id_away'], axis = 1)\
                              .sort_values(['game_id']).reset_index(drop = True)
data_games.head()
data_games.to_csv('check_split.csv')

In [ ]:
## Add a season filter

filter_season = data_games['season_id'] >= 22019

data_games_filter = data_games[filter_season]

In [ ]:
## For joins later

header_select = [
    
                 'GAME_DATE_ADJ'
                ,'game_id'
                ,'TEAM_ID'
                ,'TEAM_NAME'
                ,'WIN_FLAG'


                ]

df_join = data_games_filter[header_select]


In [ ]:
col_spark = [
    
              'GAME_DATE_ADJ'
              ,'GAME_DATE_OFFSET'
              ,'TEAM_ID'
              ,'TEAM_NAME'
              ,'HOME_FLAG'
              ,'WIN_FLAG'
              ,'SCORE'
              ,'season_id'             

              ]

df_spark_feed = data_games_filter[col_spark]

###Window Function with PySpark

In [ ]:
## Initiate Spark session
spark_1= SparkSession.builder.appName('app_1').getOrCreate()
df_1 = spark_1.createDataFrame(df_spark_feed)

In [ ]:
## Customise Windows to apply the Window Functions to
#Window_Home = Window.partitionBy("HOME_TEAM_ID").orderBy("GAME_DATE")
#Window_Away = Window.partitionBy("VISITOR_TEAM_ID").orderBy("GAME_DATE")
Window_Team_by_Date = Window.partitionBy("TEAM_ID").orderBy("GAME_DATE_ADJ")
Window_Team_by_Season = Window.partitionBy("TEAM_ID").orderBy("season_id")
Window_Team = Window.partitionBy("TEAM_ID").orderBy("HOME_FLAG")


## Derive information from customised Windows above
df_spark = df_1.withColumn("GAME_DATE_PREV", F.lag("GAME_DATE_ADJ", 1).over(Window_Team_by_Date)) \
               .withColumn("Back_to_Back_FLAG", F.when(F.col("GAME_DATE_OFFSET") == F.col("GAME_DATE_PREV"), 1) \
               .otherwise(0)) \
               .withColumn("SCORE_AVG", F.avg("SCORE").over(Window_Team)) \
               .withColumn("SCORE_AVG_by_Season", F.avg("SCORE").over(Window_Team_by_Season)) \
               .withColumn("SCORE_STD", F.stddev("SCORE").over(Window_Team))
            #.show()

#df_spark_away = df_1.withColumn("GAME_DATE_PREV_AWAY_TEAM", F.lag("GAME_DATE", 1).over(Window_Away)) \
#                    .withColumn("Back_to_Back_FLAG", F.when(F.col("GAME_DATE_OFFSET") == F.col("GAME_DATE_PREV_AWAY_TEAM"), 1) \
#                    .otherwise(0))

In [ ]:
df = df_spark.toPandas()
#df.to_csv('check_split.csv')

In [ ]:

#pd.merge(df_home, df_join, left_on = ['GAME_DATE_PREV_HOME_TEAM', 'HOME_TEAM_ID'], right_on = ['GAME_DATE', 'HOME_TEAM_ID'], how = 'left')
df_merge = pd.merge(df, df_join, left_on = ['GAME_DATE_PREV', 'TEAM_ID'], right_on = ['GAME_DATE_ADJ', 'TEAM_ID'], how = 'left')


In [ ]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8762 entries, 0 to 8761
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   GAME_DATE_ADJ_x      8762 non-null   datetime64[ns]
 1   GAME_DATE_OFFSET     8762 non-null   datetime64[ns]
 2   TEAM_ID              8762 non-null   int64         
 3   TEAM_NAME_x          8762 non-null   object        
 4   HOME_FLAG            8762 non-null   object        
 5   WIN_FLAG_x           8762 non-null   bool          
 6   SCORE                8762 non-null   int64         
 7   season_id            8762 non-null   int64         
 8   GAME_DATE_PREV       8732 non-null   datetime64[ns]
 9   Back_to_Back_FLAG    8762 non-null   int32         
 10  SCORE_AVG            8762 non-null   float64       
 11  SCORE_AVG_by_Season  8762 non-null   float64       
 12  SCORE_STD            8762 non-null   float64       
 13  GAME_DATE_ADJ_y      8732 non-nul

In [ ]:
col = [
         'TEAM_ID'
        ,'TEAM_NAME_x'
        ,'GAME_DATE_ADJ_x'
        ,'HOME_FLAG'
        ,'WIN_FLAG_x'
        ,'WIN_FLAG_y'
        ,'SCORE_AVG'
        ,'SCORE_AVG_by_Season'
        ,'SCORE_STD'
        ,'season_id'
      ]

df_b2b_interim = df_merge[df_merge['Back_to_Back_FLAG'] == 1]
df_b2b = df_b2b_interim[col].sort_values(['TEAM_ID', 'GAME_DATE_ADJ_x']).reset_index(drop = True)

In [ ]:
df_b2b.to_csv('check.csv')

### Summary Stats

In [ ]:
## Get team names

#df_teams = data_raw_teams[['TEAM_ID', 'ABBREVIATION']]
#df_stats = pd.merge(df_b2b, df_teams, on = 'TEAM_ID', how = 'left')

In [ ]:
df_b2b.WIN_FLAG_x.mean()

0.452991452991453

In [ ]:
print(df_merge.groupby(['HOME_FLAG']).mean())
print(df_b2b.groupby(['HOME_FLAG']).mean())

                TEAM_ID  WIN_FLAG_x       SCORE     season_id  \
HOME_FLAG                                                       
Away       1.610613e+09    0.444647  111.116412  22020.501027   
Home       1.610613e+09    0.555353  113.043141  22020.501027   

           Back_to_Back_FLAG   SCORE_AVG  SCORE_AVG_by_Season  SCORE_STD  \
HOME_FLAG                                                                  
Away                0.197215  111.116412           111.792130  12.256301   
Home                0.149966  112.073237           111.776185  12.362129   

                game_id  
HOME_FLAG                
Away       2.205010e+07  
Home       2.205019e+07  
                TEAM_ID  WIN_FLAG_x   SCORE_AVG  SCORE_AVG_by_Season  \
HOME_FLAG                                                              
Away       1.610613e+09    0.399306  111.034705           111.724042   
Home       1.610613e+09    0.523592  112.087496           111.795065   

           SCORE_STD     season_id  
HOME

In [ ]:
df_b2b[df_b2b['HOME_FLAG'] == 'Away'].groupby(['TEAM_NAME_x']).mean()

,TEAM_ID,WIN_FLAG_x,SCORE_AVG,SCORE_AVG_by_Season,SCORE_STD,season_id
TEAM_NAME_x,,,,,,
Atlanta Hawks,1.610613e+09,0.259259,112.349315,112.942655,12.234825,22020.481481
Boston Celtics,1.610613e+09,0.527778,112.404110,113.164243,12.191336,22020.555556
Brooklyn Nets,1.610613e+09,0.333333,114.778523,113.857138,11.642098,22020.481481
Charlotte Hornets,1.610613e+09,0.200000,110.804054,106.839208,13.678852,22020.300000
Chicago Bulls,1.610613e+09,0.565217,110.176056,109.259239,10.739533,22020.608696
Cleveland Cavaliers,1.610613e+09,0.260870,105.728571,106.563995,11.274114,22020.434783
Dallas Mavericks,1.610613e+09,0.392857,112.278912,113.986913,12.034074,22020.678571
Denver Nuggets,1.610613e+09,0.518519,111.737931,112.830250,11.520526,22020.444444
Detroit Pistons,1.610613e+09,0.233333,106.748252,106.866185,12.524606,22020.566667


In [ ]:
df_b2b.groupby(['TEAM_NAME_x', 'HOME_FLAG'])["SCORE_AVG", "SCORE_STD"].mean()

<ipython-input-172-89b6561d5e66>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_b2b.groupby(['TEAM_NAME_x', 'HOME_FLAG'])["SCORE_AVG", "SCORE_STD"].mean()


SCORE_AVG  SCORE_STD
TEAM_NAME_x            HOME_FLAG                       
Atlanta Hawks          Away       112.349315  12.234825
                       Home       114.159170  12.351434
Boston Celtics         Away       112.404110  12.191336
                       Home       113.493197  13.905027
Brooklyn Nets          Away       114.778523  11.642098
                       Home       114.197952  11.867782
Charlotte Hornets      Away       110.804054  13.678852
                       Home       110.194444  12.627484
Chicago Bulls          Away       110.176056  10.739533
                       Home       110.657343  11.847040
Cleveland Cavaliers    Away       105.728571  11.274114
                       Home       107.604167  11.545528
Dallas Mavericks       Away       112.278912  12.034074
                       Home       112.710438  12.364550
Denver Nuggets         Away       111.737931  11.520526
                       Home       113.884354  11.845431
Detroit Pistons        Away       106.748252  12.524606
                       Home       107.364583  11.997491
Golden State Warriors  Away       110.553191  13.226826
                       Home       112.303136  12.663287
Houston Rockets        Away       111.551020  13.004311
                       Home       111.368601  13.112029
Indiana Pacers         Away       111.673469  12.241946
                       Home       112.901695  11.949746
LA Clippers            Away       111.906040  12.787421
                       Home       112.605442  13.748128
Los Angeles Lakers     Away       112.081633  12.155111
                       Home       112.852740  11.879502
Memphis Grizzlies      Away       113.744828  12.038007
                       Home       114.287671  12.128441
Miami Heat             Away       108.148649  11.445443
                       Home       109.706081  11.473393
Milwaukee Bucks        Away       115.378378  14.295428
                       Home       117.217687  13.211081
Minnesota Timberwolves Away       115.744681  11.281087
                       Home       114.251748  12.174733
New Orleans Pelicans   Away       112.027211  11.082471
                       Home       113.238908  11.072191
New York Knicks        Away       107.159722  13.731523
                       Home       108.148789  13.280965
Oklahoma City Thunder  Away       106.923611  12.826480
                       Home       108.744027  13.181531
Orlando Magic          Away       105.891892  11.317397
                       Home       106.566102  11.885002
Philadelphia 76ers     Away       110.445205  12.206291
                       Home       112.208191  12.079687
Phoenix Suns           Away       113.344828  11.726264
                       Home       114.309524  11.562685
Portland Trail Blazers Away       110.546667  12.401335
                       Home       112.691525  12.789096
Sacramento Kings       Away       112.417808  12.344345
                       Home       113.565068  12.882467
San Antonio Spurs      Away       112.013605  13.055197
                       Home       112.508591  12.235721
Toronto Raptors        Away       110.195946  11.957782
                       Home       111.363946  11.689974
Utah Jazz              Away       112.671141  12.387572
                       Home       114.329932  11.664683
Washington Wizards     Away       111.716216  12.253580
                       Home       112.784983  13.884979

In [ ]:
pd.set_option('display.max_rows', None)
df_b2b.groupby(['TEAM_NAME_x', 'season_id'])["SCORE_AVG_by_Season"].mean()

TEAM_NAME_x             season_id
Atlanta Hawks           22019        111.761194
                        22020        112.762590
                        22021        113.199095
                        22022        114.159170
Boston Celtics          22019        113.652778
                        22020        113.138889
                        22021        112.637168
                        22022        113.493197
Brooklyn Nets           22019        111.777778
                        22020        115.173611
                        22021        114.349558
                        22022        114.197952
Charlotte Hornets       22019        102.876923
                        22020        106.335766
                        22021        109.703196
                        22022        110.194444
Chicago Bulls           22019        106.846154
                        22020        108.861314
                        22021        109.890411
                        22022        110.657343
Clevel

In [ ]:
#%%shell
#jupyter nbconvert --to html /content/NBA_data_analytics.ipynb